# Pregunta 3: Implementación

## Parte 1
Se implementará la clase `RegresionBayesianaEmpirica`, que heredará las clases [`BaseEstimator`](https://scikit-learn.org/stable/modules/generated/sklearn.base.BaseEstimator.html) y [`RegressorMixin`](http://scikit-learn.org/stable/modules/generated/sklearn.base.RegressorMixin.html), que son clases del paquete [`sklearn.base`](https://scikit-learn.org/stable/modules/classes.html#module-sklearn.base)

### Importaciones

In [ ]:
import numpy as np
from sklearn.base import BaseEstimator, RegressorMixin

### Implementación de la clase `RegresionBayesianaEmpirica`

In [ ]:
class RegresionBayesianaEmpirica(BaseEstimator, RegressorMixin):
    """
    `RegresionBayesianaEmpirica` es una clase que hereda de `sklearn.base.BaseEstimator` y 
    `sklearn.base.RegressorMixin` el cual implementa la heurística enunciada en el 
    informe de la tarea para aproximar los hiperparámetros óptimos de alpha y beta.
    """
    def __init__(self, alpha_0, beta_0, tol=1e-5, maxiter=200):
        self.alpha = alpha_0
        self.beta = beta_0
        self.__tol = tol
        self.__maxiter = maxiter
        
    def get_posterior(self, X, y, alpha, beta):
        """
        Recibe una matriz de observaciones 'X' (de dimensiones N x d), el vector de 
        etiquetas 'y' (de dimensión N) y los hiperparámetros 'alpha' y 'beta'.
        
        Retorna una tupla (m_N, S_N), dónde 'm_N' corresponde al vector de medias y 
        'S_N' corresponde a la matriz de covarianzas de la posterior de 'w'.
        """
        # Se ocupa P3-1
        # dimensiones
        N, d = X.shape
        
        # Matriz de covarianzas [d x d]
        S_N_inv = alpha*np.eye(d) + beta*X.T@X
        S_N = np.linalg.inv(S_N_inv) 
        
        # Vector de medias [d]
        m_N = beta * (S_N@X.T@y)
        
        return (m_N, S_N)
    
    def fit(self, X, y):
        """
        Recibe una matriz de observaciones 'X' (de dimensiones N x d) 
        y un vector de etiquetas 'y' (de dimensión N).
        
        Ajusta los valores de 'alpha' y 'beta'.
        """
        # Seteamos atributos de los datos y dimensiones
        self.X = X
        self.y = y
        self.N, self.d = self.X.shape
        
        maxiter_alcanzado = True # Variable para saber si la 
                                 # máxima iteración ha sido cumplida o no
        
        for _ in range(self.__maxiter):
            # Seteamos alpha_0 y beta_0 
            alpha_0, beta_0 = self.alpha, self.beta
            
            # Calculamos los valores propios de beta * X^t * X
            lamb, _ = np.linalg.eig(beta_0 * self.X.T@self.X)
            
            # Calculamos gamma
            gamma = sum([lamb[i] / (alpha_0 + lamb[i]) for i in range(self.d)])
            
            # Calculamos m_N
            m_N, S_N = self.get_posterior(self.X, self.y, alpha_0, beta_0)
            
            # Calculamos el siguiente valor de alpha
            alpha_1 = gamma / np.inner(m_N, m_N)
            
            # Y calculamos el siguiente valor de beta
            beta_1_inv = (1 / (self.N-gamma)) \
                     * sum([(self.y[i] - np.inner(m_N, self.X[i]))**2 for i in range(self.N)])
            beta_1 = 1 / beta_1_inv
            
            # Seteamos estos nuevos valores para alpha y beta
            self.alpha, self.beta = alpha_1, beta_1
            
            # Comparamos para saber si ya se cumplió la condición de cercanía
            if (abs(alpha_0 - alpha_1) <= self.__tol
               and abs(beta_0 - beta_1) <= self.__tol):
                print('tolerancia alcanzada')
                maxiter_alcanzado = False
                break
        
        if maxiter_alcanzado:
            print('alcanzado numero max de iter')
        
        return self
            
    def predict(self, X_, return_std=False)->tuple:
        """
        Recibe una matriz de observaciones 'X_' (de dimensiones N' x d).
        
        Retorna una tupla (y_, y_std), en dónde 'y_' corresponde al vector de medias y 'y_std' 
        corresponde al vector de desviaciones estándar asociadas a las observaciones de 'X_'.
        Haciendo notar que esto ocurre solo sí 'return_std' es True. En caso contrario, solo retorna 
        la tupla (y_,).
        
        Se ocupa la distribución predictiva posterior para estas predicciones.
        """
        # Se ocupa P3-2
        # Dimensiones necesarias
        N_, d = X_.shape
        m_N, S_N = self.get_posterior(self.X, self.y, self.alpha, self.beta)
        
        y_ = []
        y_std = []
        for i_ in range(N_):
            x_i = X_[i_]
            y_i = np.inner(m_N, x_i)
            y_std_i = 1/self.beta + x_i.T@S_N@x_i
            
            y_.append(y_i)
            y_std.append(y_std_i)
            
        if return_std:
            return (y_, y_std)
        else:
            return (y_,)


### Prueba de la clase

Se creará un data set de prueba para experimentar si la clase `RegresionBayesianaEmpirica` funciona. 

In [ ]:
# Módulos que quizas sean necesarios
import matplotlib.pyplot as plt
import matplotlib.cm as cm
from scipy.stats import norm
from scipy.stats import multivariate_normal
import seaborn as sns # pretty plots
import numpy as np

#### Constantes

In [ ]:
N, d = 5000, 3
alpha_real, beta_real = 1, 2.5

#### Vector de pesos

In [ ]:
# v_cero = [0 for i in range(d)]
# M_bet_I = (1 / beta_real) * np.eye(d)
# w = np.random.multivariate_normal(v_cero, M_bet_I)
w = np.array([3, 4, 5])

#### Matriz X y vector y

In [ ]:
X = np.array([
    [1] + [np.random.uniform(-5, 5) for _ in range(d-1)] for _ in range(N)
])

y = np.array([
    float(np.inner(w, X[i]) + np.random.normal(0, alpha_real)) for i in range(N)
])

#### Prueba de la clase

In [ ]:
alpha_falso, beta_falso = 50000, 2.5

rbe = RegresionBayesianaEmpirica(alpha_falso, beta_falso)
rbe.fit(X, y)

#### Predicción

In [ ]:
X_ = np.array([
    [8, 2, 5],
    [2, 4, 9]
])
rbe.predict(X_, return_std=True)

In [ ]:
np.inner(X_[0], w), np.inner(X_[1], w)

In [ ]:
rbe.predict?

In [ ]:
lam, _ = np.linalg.eig(beta_falso * X.T@X)
beta_falso * X.T@X